In [ ]:
!pip install -q transformers scikit-learn pandas torch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 69.8 MB/s eta 0:00:00


In [32]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import cosine_distances

In [ ]:
# to import files from googledrive
from google.colab import drive


drive.mount('/content/gdrive/', force_remount=True)
sample_size_dir = 'gdrive/My Drive/Sample size for NLP'

Mounted at /content/gdrive/


In [47]:
# diagnosis = '5849'
diagnosis = 'original'
description = 'hypertension'

In [48]:
# df = pd.read_csv(sample_size_dir+'/data_jaya/'+diagnosis+'_combined_data_1000.csv')
df = pd.read_csv(sample_size_dir+'/Sample_Size_Saniya/12k_data_for_classifier.csv')
df.head()

,Unnamed: 0,subject_id,icd9_code,gender,dob,ethnicity,text,no_of_docs,label
0,0,1738,4019,F,2090-12-20 00:00:00,WHITE,CONDITION UPDATE\nD: PLEASE SEE CAREVUE FOR SP...,48,1
1,1,23055,4019,F,2044-10-15 00:00:00,UNKNOWN/NOT SPECIFIED,MSICU NSG ADMIT NOTE 1400-1900\n\n60 YO woman ...,50,1
2,2,21457,4019,F,2059-06-19 00:00:00,BLACK/AFRICAN AMERICAN,"[**2121-7-5**] 5:18 PM\n TOE(S), 2+ VIEW RIGHT...",41,1
3,3,1048,4019,M,2116-04-12 00:00:00,WHITE,Sinus rhythm. Left atrial abnormality. Promine...,18,1
4,4,18041,4019,M,2133-06-05 00:00:00,WHITE,[**2196-8-1**] 7:07 PM\n CTA CHEST W&W/O C &RE...,42,1


In [35]:
# Load BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('cambridgeltl/SapBERT-from-PubMedBERT-fulltext') # bert-base-uncased, cambridgeltl/SapBERT-from-PubMedBERT-fulltext
model = BertModel.from_pretrained('cambridgeltl/SapBERT-from-PubMedBERT-fulltext') # bert-base-uncased, cambridgeltl/SapBERT-from-PubMedBERT-fulltext

In [36]:
# Function to get BERT embeddings
def get_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

In [37]:
# Function to calculate cosine similarity between two embeddings
def calculate_cosine_similarity(embedding1, embedding2):
    return cosine_similarity([embedding1], [embedding2])[0][0]

In [49]:
# Sentence to compare with
comparison_sentence = "This patient has been diagnosed with "+description
comparison_sentence

'This patient has been diagnosed with hypertension'

In [50]:
# Get BERT embedding for the comparison sentence
comparison_embedding = get_bert_embeddings(comparison_sentence)

In [51]:
# Calculate cosine similarity for each row and add as a new column
# df['cosine score'] = df['TEXT'].apply(lambda x: calculate_cosine_similarity(get_bert_embeddings(x), comparison_embedding))
df['cosine score'] = df['text'].apply(lambda x: calculate_cosine_similarity(get_bert_embeddings(x), comparison_embedding))


In [52]:
df.tail()

,Unnamed: 0,subject_id,icd9_code,gender,dob,ethnicity,text,no_of_docs,label,cosine score
11995,11995,1940,2762,M,1814-10-20 00:00:00,WHITE,Respiratory Care:\nPatient received from ED S/...,50,0,0.254695
11996,11996,2168,V290,M,2192-10-02 00:00:00,HISPANIC OR LATINO,"Neonatology Attending\nExam AF soft,flat, slee...",50,0,0.305955
11997,11997,2832,41401,M,2068-10-05 00:00:00,WHITE,Sinus rhythm. Borderline P-R interval prolonga...,35,0,0.183793
11998,11998,595,340,F,2120-12-09 00:00:00,WHITE,[**2166-5-13**] 11:36 AM\n ERCP S&I ([**Numeri...,26,0,0.287751
11999,11999,1393,76519,M,2108-10-04 00:00:00,WHITE,Nursing Note\nInfant passed car seat test 90 m...,32,0,0.198658


In [53]:
df.to_csv(sample_size_dir+'/data_jaya/cosine_scores/'+diagnosis+'_combined_data_1000_w_cosine_sapbert.csv')

get cosine distance (1-cosine similarity)

In [54]:
def calculate_cosine_distance(similarity):
    return 1 - similarity

In [55]:
# Add cosine distance column
df['cosine_distance'] = df['cosine score'].apply(calculate_cosine_distance)


In [56]:
df.head()

,Unnamed: 0,subject_id,icd9_code,gender,dob,ethnicity,text,no_of_docs,label,cosine score,cosine_distance
0,0,1738,4019,F,2090-12-20 00:00:00,WHITE,CONDITION UPDATE\nD: PLEASE SEE CAREVUE FOR SP...,48,1,0.205524,0.794476
1,1,23055,4019,F,2044-10-15 00:00:00,UNKNOWN/NOT SPECIFIED,MSICU NSG ADMIT NOTE 1400-1900\n\n60 YO woman ...,50,1,0.282458,0.717542
2,2,21457,4019,F,2059-06-19 00:00:00,BLACK/AFRICAN AMERICAN,"[**2121-7-5**] 5:18 PM\n TOE(S), 2+ VIEW RIGHT...",41,1,0.342456,0.657544
3,3,1048,4019,M,2116-04-12 00:00:00,WHITE,Sinus rhythm. Left atrial abnormality. Promine...,18,1,0.245747,0.754253
4,4,18041,4019,M,2133-06-05 00:00:00,WHITE,[**2196-8-1**] 7:07 PM\n CTA CHEST W&W/O C &RE...,42,1,0.333973,0.666027


In [57]:
df.to_csv(sample_size_dir+'/data_jaya/cosine_scores/'+diagnosis+'_combined_data_1000_w_cosine_sapbert.csv')

In [ ]:
# Generate embeddings for each row in the dataframe
df['embedding'] = df['TEXT'].apply(get_bert_embeddings)

In [ ]:
# Separate embeddings by label
embeddings_label_0 = df[df['label'] == 0]['embedding'].tolist()
embeddings_label_1 = df[df['label'] == 1]['embedding'].tolist()

In [ ]:
# Calculate pairwise cosine distances between the two groups
'''
distances = []

for i, emb0 in enumerate(embeddings_label_0):
    for j, emb1 in enumerate(embeddings_label_1):
        distance = cosine_distances([emb0], [emb1])[0][0]
        distances.append({
            'text_label_0': df[df['label'] == 0].iloc[i]['TEXT'],
            'text_label_1': df[df['label'] == 1].iloc[j]['TEXT'],
            'cosine_distance': distance
        })
'''

In [ ]:
# Create a DataFrame to store the pairwise distances
# distance_df = pd.DataFrame(distances)
# distance_df.head()

,text_label_0,text_label_1,cosine_distance
0,Admission Date: [**2100-8-29**] Dischar...,Admission Date: [**2158-6-28**] ...,0.312236
1,Admission Date: [**2100-8-29**] Dischar...,Admission Date: [**2185-1-24**] ...,0.284139
2,Admission Date: [**2100-8-29**] Dischar...,Admission Date: [**2185-11-29**] ...,0.346941
3,Admission Date: [**2100-8-29**] Dischar...,Admission Date: [**2148-1-2**] D...,0.291663
4,Admission Date: [**2100-8-29**] Dischar...,Admission Date: [**2131-2-23**] ...,0.285450


In [ ]:
# distance_df.to_csv(sample_size_dir+'/data_jaya/'+diagnosis+'_cosine_distances_bw_labels.csv')

# word2vec

In [ ]:
!pip install -q gensim scikit-learn


In [ ]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [ ]:
df = pd.read_csv(sample_size_dir+'/data_jaya/51881_combined_data_1000.csv')
df.head()

In [ ]:
# Function to preprocess text
def preprocess_text(text):
    return text.lower().split()

In [ ]:
# Prepare sentences for Word2Vec training
sentences = df['TEXT'].apply(preprocess_text).tolist()

In [ ]:
# Train a Word2Vec model
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

In [ ]:
# Function to get Word2Vec embeddings for a sentence
def get_word2vec_embeddings(sentence):
    words = preprocess_text(sentence)
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    if not word_vectors:
        return np.zeros(model.vector_size)
    return np.mean(word_vectors, axis=0)

In [ ]:
# Sentence to compare with
comparison_sentence = "This patient has been diagnosed with acute respiratory failure"

In [ ]:
# Get Word2Vec embedding for the comparison sentence
comparison_embedding = get_word2vec_embeddings(comparison_sentence)

In [ ]:
# Calculate cosine similarity for each row and add as a new column
df['cosine score'] = df['TEXT'].apply(lambda x: cosine_similarity([get_word2vec_embeddings(x)], [comparison_embedding])[0][0])

In [ ]:
df.head()


In [ ]:
df.to_csv(sample_size_dir+'/data_jaya/51881_combined_data_1000_w_cosine_w2v.csv')